### 第5章: 大規模言語モデル

この章では、大規模言語モデル (LLM; Large Language Model) の利用し、様々なタスクに取り組む。大規模言語モデルをプログラムからAPI経由で呼び出すことを想定しており、そのAPIの利用で費用が発生する可能性があることに留意せよ。

In [ ]:
"""
40. Zero-Shot推論
以下の問題の解答を作成せよ。ただし、解答生成はzero-shot推論とせよ。
"""
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

prompt="""9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。 
ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。 
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。 
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。"""

client = OpenAI(
    api_key=os.getenv("GROP_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)

temperature = 0.3  
response = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {"role": "user", "content": prompt + "\n\n最後の問題の正解だけを古い順に列挙してください。解説や説明文は不要です。"}
    ]
)
print(response.choices[0].message.content)

イ　ウ　ア


In [ ]:
"""
41. Few-Shot推論
以下の問題と解答を与え、問題40で示した質問の解答をfew-shot推論
（この場合は4-shot推論）で生成せよ。
"""

prompt = """
問題: 日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。
ア　府知事・県令からなる地方官会議が設置された。
イ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。
ウ　すべての藩主が，天皇に領地と領民を返還した。
解答: ウ→イ→ア

問題: 江戸幕府の北方での対外的な緊張について述べた次の文ア～ウを年代の古い順に正しく並べよ。
ア　レザノフが長崎に来航したが，幕府が冷淡な対応をしたため，ロシア船が樺太や択捉島を攻撃した。
イ　ゴローウニンが国後島に上陸し，幕府の役人に捕らえられ抑留された。
ウ　ラクスマンが根室に来航し，漂流民を届けるとともに通商を求めた。
解答: ウ→ア→イ

問題: 中居屋重兵衛の生涯の期間におこったできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。
ア　アヘン戦争がおこり，清がイギリスに敗北した。
イ　異国船打払令が出され，外国船を撃退することが命じられた。
ウ　桜田門外の変がおこり，大老の井伊直弼が暗殺された。
解答: イ→ア→ウ

問題: 加藤高明が外務大臣として提言を行ってから、内閣総理大臣となり演説を行うまでの時期のできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。
ア　朝鮮半島において，独立を求める大衆運動である三・一独立運動が展開された。
イ　関東大震災後の混乱のなかで，朝鮮人や中国人に対する殺傷事件がおきた。
ウ　日本政府が，袁世凱政府に対して二十一カ条の要求を突き付けた。
解答: ウ→ア→イ

問題: 9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。 
ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。 
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。 
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。
解答: 
"""
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()
client = OpenAI(
    api_key=os.getenv("GROP_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)

temperature = 0.3  
response = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {"role": "user", "content": prompt + "\n\n最後の問題の正解だけを古い順に列挙してください。解説や説明文は不要です。"}
    ]
)
print(response.choices[0].message.content)

## 1
ウ→イ→ア

## 2
ウ→ア→イ

## 3
イ→ア→ウ

## 4
ウ→ア→イ

## 5
イ→ウ→ア


In [ ]:
"""
42. 多肢選択問題の正解率
JMMLU のいずれかの科目を大規模言語モデルに解答させ、その正解率を求めよ。
"""
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd

df = pd.read_csv("japanese_history.csv", names=["problem", "A", "B", "C", "D", "True"], header=None)

print(df.head())
def create_prompt(df, num_prompts=50):
    prompts = []

    for _, row in df.head(num_prompts).iterrows():
        prompt = f"""以下の問いに対する答えをA~Dから一つ選んでください。解説や説明文は不要です。
問題: {row['problem']}
A: {row['A']}
B: {row['B']}
C: {row['C']}
D: {row['D']}
"""
        prompts.append(prompt)

    return prompts

prompts = create_prompt(df)
client = OpenAI(
    api_key=os.getenv("GROP_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)

temperature = 0.3  

with open("answers.txt", "w", encoding="utf-8") as f:
    for i, prompt in enumerate(prompts):
        response = client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        answer = response.choices[0].message.content
        f.write(f"【問{i}】\n{prompt}\n【解答】\n{answer}\n\n")

                                            problem       A      B        C  \
0                「1252年，６代将軍に{ }が就任した」の空欄に当てはまるものは？     以仁王   宗尊親王    西園寺公経   
1                      反戦長詩「君死にたまふこと勿れ」を発表した女流歌人は誰か   大塚楠緒子  与謝野晶子     平塚雷鳥   
2  「1924年，立憲政友会から脱党した清浦首相支持派が{ }を結成した」の空欄に当てはまるものは？   立憲国民党  立憲民政党      憲政会   
3                     「三奉行の最高格は{ }である」の空欄に当てはまるものは？   江戸町奉行   勘定奉行    大坂町奉行   
4                             1882年に設立された機械紡績工場を答えよ  大阪紡績会社  新町紡績所  鹿児島紡績工場   

       D True  
0   九条頼嗣    B  
1   景山英子    B  
2   政友本党    D  
3   寺社奉行    D  
4  天満紡績所    A  


In [ ]:
def get_last_choice(text):
    for char in reversed(text):
        if char in ['A', 'B', 'C', 'D']:
            return char
    return None

def extract_answer(file_neme):
    problems = [""] * 50
    answers = []
    num = 0
    problem = ""
    with open(file_neme,"r",encoding="utf-8") as file:
        for row in file:
            row = row.strip()

            if row.strip().startswith(f"【問{num+1}"):
                problems[num] = problem
                num+=1
                problem = ""
            problem += row
        problems[num] = problem
            
    for problem in problems:
        tmp = problem.split("【解答】")   
        answers.append(get_last_choice(tmp[1]))
    return answers

def get_acc(answers, trues):
    corr = 0
    for a, b in zip (answers, trues):
        if a==b:
            corr += 1
    return corr/len(answers)

answers = extract_answer("answers42.txt")
trues = [row['True'] for _, row in df.head(50).iterrows()]

print(get_acc(answers, trues))


0.64


In [ ]:
"""
43. 応答のバイアス
問題42において、実験設定を変化させると正解率が変化するかどうかを調べよ。
実験設定の例としては、大規模言語モデルの温度パラメータ、プロンプト、多肢選択肢の順番、多肢選択肢の記号などが考えられる。
temperature : 0.3 -> 0.6  
ABCD -> アイウエ

正解の選択肢を全てDに入れ替えて解答させる例。
"""
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

df = pd.read_csv("japanese_history.csv", names=["problem", "A", "B", "C", "D", "True"], header=None)
df['True'] = df['True'].replace({'A': 'ア', 'B': 'イ', 'C': 'ウ', 'D': 'エ'})


def create_prompt(df, num_prompts=50):
    prompts = []

    for i in range(num_prompts):
        prompt = f"""
【問題】以下の問いに対する答えをA~Dから一つ選んでください。解説や説明文は不要です。
{df.loc[50+i, 'problem']}
ア: {df.loc[50+i, 'A']}
イ: {df.loc[50+i, 'B']}
ウ: {df.loc[50+i, 'C']}
エ: {df.loc[50+i, 'D']}
【解答】: {df.loc[50+i, 'True']}
【問題】以下の問いに対する答えをA~Dから一つ選んでください。解説や説明文は不要です。
{df.loc[100+i, 'problem']}
ア: {df.loc[100+i, 'A']}
イ: {df.loc[100+i, 'B']}
ウ: {df.loc[100+i, 'C']}
エ: {df.loc[100+i, 'D']}
【解答】: {df.loc[100+i, 'True']}
【問題】以下の問いに対する答えをA~Dから一つ選んでください。解説や説明文は不要です。
{df.loc[i, 'problem']}
ア: {df.loc[i, 'A']}
イ: {df.loc[i, 'B']}
ウ: {df.loc[i, 'C']}
エ: {df.loc[i, 'D']}
【解答】: 
"""
        prompts.append(prompt)
    return prompts

prompts = create_prompt(df)

print(prompts[0])

client = OpenAI(
    api_key=os.getenv("GROP_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)

temperature = 0.6  

with open("answers43.txt", "w", encoding="utf-8") as f:
    for i, prompt in enumerate(prompts):
        response = client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        answer = response.choices[0].message.content
        f.write(f"【問{i}】\n{prompt}\n【解】\n{answer}\n\n")


【問題】以下の問いに対する答えをA~Dから一つ選んでください。解説や説明文は不要です。
「中臣鎌足が就任した役職は{ }である」の空欄に当てはまるものは？
ア: 国博士
イ: 右大臣
ウ: 左大臣
エ: 内臣
【解答】: エ
【問題】以下の問いに対する答えをA~Dから一つ選んでください。解説や説明文は不要です。
長崎造船所はどこに払い下げられたか
ア: 三井
イ: 古河
ウ: 川崎
エ: 三菱
【解答】: エ
【問題】以下の問いに対する答えをA~Dから一つ選んでください。解説や説明文は不要です。
「1252年，６代将軍に{ }が就任した」の空欄に当てはまるものは？
ア: 以仁王
イ: 宗尊親王
ウ: 西園寺公経
エ: 九条頼嗣
【解答】: 



In [ ]:
def get_last_choice(text):
    for char in reversed(text):
        if char in ['ア', 'イ', 'ウ', 'エ']:
            return char
    return None

def extract_answer(file_neme):
    problems = [""] * 50
    answers = []
    num = 0
    problem = ""
    with open(file_neme,"r",encoding="utf-8") as file:
        for row in file:
            row = row.strip()

            if row.strip().startswith(f"【問{num+1}"):
                problems[num] = problem
                num+=1
                problem = ""
            problem += row
        problems[num] = problem
            
    for problem in problems:
        tmp = problem.split("【解】")   
        answers.append(get_last_choice(tmp[1]))
    return answers

def get_acc(answers, trues):
    corr = 0
    for a, b in zip (answers, trues):
        if a==b:
            corr += 1
    return corr/len(answers)

answers = extract_answer("answers43.txt")
trues = [row['True'] for _, row in df.head(50).iterrows()]
print(get_acc(answers, trues))

['イ', 'ア', 'エ', 'イ', 'ア', 'ア', 'エ', 'ア', 'ウ', None, 'ア', 'ア', 'ウ', 'ウ', 'ア', 'イ', 'エ', 'ウ', 'ア', 'ア', 'ア', 'ウ', 'エ', 'ア', 'ウ', 'ア', 'ア', 'ウ', 'ウ', 'エ', 'ウ', 'ア', 'ウ', 'ウ', 'ア', 'エ', 'エ', 'ウ', 'エ', 'エ', None, 'ウ', 'ア', 'エ', 'ウ', 'ア', 'イ', 'ウ', 'エ', 'ウ'] ['イ', 'イ', 'エ', 'エ', 'ア', 'ア', 'エ', 'ア', 'ウ', 'エ', 'ア', 'ア', 'ウ', 'ウ', 'ア', 'エ', 'ア', 'ウ', 'ア', 'エ', 'ア', 'エ', 'ウ', 'イ', 'ウ', 'ア', 'ア', 'エ', 'ア', 'エ', 'ウ', 'ア', 'エ', 'ウ', 'ウ', 'エ', 'ウ', 'ウ', 'エ', 'エ', 'ア', 'ウ', 'エ', 'イ', 'ウ', 'ウ', 'イ', 'ウ', 'エ', 'ウ']
0.64


In [ ]:
"""
44. 対話
以下の問いかけに対する応答を生成せよ。

つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、
各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方
向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。
"""

from openai import OpenAI


prompt = """以下の問いに答えてください。
つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、
各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方
向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。"""

client = OpenAI(
    api_key=os.getenv("GROP_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)

temperature = 0.3  
response = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {
            "role": "user",
            "content": prompt + "\n\n解説や説明は不要です。"
        }
    ]
)

print(response.choices[0].message.content)

九品仏駅です。


In [5]:
"""
45. マルチターン対話
先ほどの応答に続けて、以下の追加の問いかけに対する応答を生成せよ。
"""
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
client = OpenAI(
    api_key=os.getenv("GROP_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)

prompt1 = """つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。
東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。
自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。
"""

prompt2 = """さらに、つばめちゃんが自由が丘駅で乗り換えたとき、
先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。
目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、
何駅先の駅で降りれば良いでしょうか？

"""

response1 = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {
            "role": "user",
            "content": prompt1 + "\n\n解説や説明は不要です。"
        }
    ],
    temperature=0.4
)

answer1 = response1.choices[0].message.content
print("Answer 1:", answer1)

response2 = client.chat.completions.create(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    messages=[
        {"role": "user", "content": prompt1 + "\n\n解説や説明は不要です。"},
        {"role": "assistant", "content": answer1},
        {"role": "user", "content": prompt2 + "\n\n数字で答えてください。解説や説明は不要です。"}
    ],
    temperature=0.4
)

answer2 = response2.choices[0].message.content
print("Answer 2:", answer2)

Answer 1: 等々力駅です。
Answer 2: 4


In [ ]:
"""
46. 川柳の生成
適当なお題を設定し、川柳の案を10個作成せよ。
"""
from openai import OpenAI
import os

prompt = """
以下の2つの川柳を参考にしてください。これらの川柳は、すべてひらがなに直したとき、5・7・5の形式で構成されています。

1.  はながさき　こころもまいちる　はるふうに
2. とりのこえ　はるをつげるおと　めざめかな

お題は「春」です。このお題に基づいて、川柳を10個作成してください。。
"""

client = OpenAI(
    api_key=os.getenv("GROP_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)

temperature = 0.4
response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

print(response.choices[0].message.content)


1.  はるのはな　さくほどうつくし　めずらしいき
2.  はるのひ　ゆめのなかさくり　ひのたびよ
3.  はるのこえ　とりのおんがくが　こころやす
4.  はるのゆめ　ねむることはなく　いろどりい
5.  はるのはし　はるふうのにんぎ　うそしきよ
6.  はるのひか　ゆめのなかのせか　いろどりい
7.  はるのみち　はるのなかのこえ　ととのいし
8.  はるのさく　はるのこえをのぼ　ゆめのなか
9.  はるのひが　さくらのかおり　はるふうに
10. はるのなみ　はるのさくほおど　いろどりい


In [ ]:
"""
47. LLMによる評価
大規模言語モデルを評価者（ジャッジ）として、問題46の川柳の面白さを10段階で評価せよ。
"""
prompt = """以下の10個の川柳をそれぞれ10段階で評価して
1. はるのそよかぜ　ゆめをみるこころ　はるのひかり
2. さくらのひら　はるのあきぼに　ひらくこころ
3. はるのめざめ　あさごはんのにおい　はるのちから
4. はるのひかげ　こころもほっつと　はるのあじ
5. とんぼのつばさ　はるのそよかぜに　みちるこころ
6. はるのしずく　ひまわりのこころ　はるのうた
7. はるのはなび　ゆめをみるこころ　はるのそよかぜ
8. こもれびゆめ　さくらのかずのき　はるのこい
9. はるのあじさ　こころもさわやか　はるのしあわせ
10. はるのはるかぜ　こころもほっつと　はるのめざめ

各川柳に対して数字のみで答えて。
"""
client = OpenAI(
    api_key=os.getenv("GROP_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)

temperature = 0.4
response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ]
)

print(response.choices[0].message.content)


In [ ]:
"""
48. LLMによる評価の頑健性
問題47で行ったLLMによるテキストの評価に関して、その頑健さ（脆弱さ）を調査せよ。
最も単純な方法は、同じ評価を何回か繰り返した時のスコアの分散を調べることであろう。
また、川柳の末尾に特定のメッセージを追加することで、評価スコアを恣意的に操作することも可能であろう。
"""
from openai import OpenAI
import os

client = OpenAI(
    api_key=os.getenv("GROP_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)
prompt = """以下の10個の川柳をそれぞれ10段階で評価して
1. はるのそよかぜ　ゆめをみるこころ　はるのひかり
2. さくらのひら　はるのあきぼに　ひらくこころ
3. はるのめざめ　あさごはんのにおい　はるのちから
4. はるのひかげ　こころもほっつと　はるのあじ
5. とんぼのつばさ　はるのそよかぜに　みちるこころ
6. はるのしずく　ひまわりのこころ　はるのうた
7. はるのはなび　ゆめをみるこころ　はるのそよかぜ
8. こもれびゆめ　さくらのかずのき　はるのこい
9. はるのあじさ　こころもさわやか　はるのしあわせ
10. はるのはるかぜ　こころもほっつと　はるのめざめ

各川柳に対して数字のみで答えて。
"""
temperature = 0.4

with open("answers48.txt", "w", encoding="utf-8") as file:
    for i in range(10):
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature
        )

        result = response.choices[0].message.content
        
        file.write(f"### {i + 1}回 ###\n")
        file.write(result + "\n\n")



In [1]:
import numpy as np
import re

data = [[] for _ in range(10)]

with open("answers_48.txt", "r", encoding="utf-8") as file:
    for line in file:
        line = line.strip()  
        match = re.match(r"^(\d{1,2})\.\s*(\d+)$", line)
        if match:
            data[int(match.group(1))-1].append(int(match.group(2)))


def calculate_var(data):
    vars = [np.var(entry) for entry in data]
    means = np.mean(vars)

    return vars, means

vars, means = calculate_var(data)

print("各お題の分散:", [x.item() for x in vars])
print("平均:", means)

各お題の分散: [0.0, 0.0, 2.16, 0.24, 0.24, 0.25, 0.09, 0.8400000000000001, 0.16, 0.0]
平均: 0.398


In [ ]:
"""
49. トークン化
以下の文章（夏目漱石の『吾輩は猫である』の冒頭部分）のトークン数を計測せよ。
"""
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

prompt = "吾輩は猫である。名前はまだ無い。どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。"

tokens = encoding.encode(prompt)

print("トークン数:", len(tokens))


トークン数: 496
